In [1]:
import numpy as np
from sympy import *
import typing
from typing import Callable
from typing import Any

In [97]:
def fem1d(n : int, interval : Any, c_func : Callable[[float, float], float], f_func : Callable[[float, float], float], bc = None):
    """
    n : n等分
    interval : 1d区间
    c_func : 自定义的计算定积分的函数 ,接收两个参数(下限，上限)
    f_func : 自定义的计算定积分的函数 ,接收两个参数(下限，上限，插值点)
    retrun : Ax=b --> x = inv(A) * b
    """
    def stiffmat(n : int, interval : Any, c_func : Callable[[float, float], float] ):
        
        A = np.zeros(shape = (n+1, n+1))
        u = interval[1]
        b = interval[0]
        h = (u - b) / n
        h_ = 1 / (h**2)
        for j in range(0,n+1):
            #print(j)
            x_j = b + h * j
            if j < n:
                A[j+1][j] = - h_ * c_func(x_j, x_j + h) 
            if j > 0:
                A[j-1][j] = - h_ * c_func(x_j - h, x_j) 
            if 0 < j and j < n:
                A[j][j] = h_ * (c_func(x_j - h, x_j) + c_func(x_j, x_j + h))
        if bc is not None :
            A[0][0] = 1.0
            A[n][n] = 1.0
        else :
            A[0][0] = h_ * c_func(b, b+h)
            A[n][n] = h_ * c_func(u-h, u)
        return A

    def loadvec(n : int, interval : Any, f_func : Callable[[float, float], float]):
        vec = np.zeros(shape = (n+1, 1))
        u = interval[1]
        b = interval[0]
        h = (u - b) / n
        for i in range(1, n):
            x_i = b + h * i
            vec[i] = f_func(x_i - h, x_i, x_i - h) / h - f_func(x_i, x_i + h, x_i+h) / h
        if bc is not None:
            vec[0] = bc[0]
            vec[n] = bc[1]
        else:
            vec[0] = -1.0 * f_func(b, b + h, b + h)
            vec[n] = f_func(u - h, u, u - h)
        return vec 

    A = stiffmat(n, interval, c_func)
    lambdas, V = np.linalg.eig(A.T@A)
    if (np.any(np.isclose(lambdas, 0))):
        print(f"A是奇异矩阵, 有非常接近0的特征值:{lambdas[np.isclose(lambdas, 0)]}")
        return 
    b = loadvec(n, interval, f_func)
    print(A)
    return np.linalg.solve(A,b)


In [98]:
def my_c(b : float, u : float):
    x = symbols('x')
    return integrate(cos(x), (x, b, u))
def my_f(b : float, u : float, i : float):
    x = symbols('x')
    return integrate(x * (x - i), (x, b ,u))
n = 2
interval = [0.0, 1.0]
fem1d(n, interval, my_c, my_f)

A是奇异矩阵, 有非常接近0的特征值:[-6.50902399e-16]
[-6.50902399e-16]
